In [63]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json

In [64]:
from os import listdir
from os.path import isfile, join

mypath = '/Users/data/Documents/actas/datos/'
files = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [65]:
files

['project-24-at-2021-08-20-23-09-37a57061.json',
 'project-22-at-2021-08-20-23-11-8bf5dc2e.json',
 'project-23-at-2021-08-20-23-10-1f5635ab.json',
 'project-26-at-2021-08-20-23-06-62aca1f2.json',
 'project-25-at-2021-08-20-23-08-c5ee9d69.json',
 'project-20-at-2021-08-20-23-13-69739800.json']

In [66]:
# quiero saber cuáles son todos los campos posibles para hacer columnas en el df

# agrego estos a mano porque en el loop no salen solos
campos = ['id', 'documento', 'pagina', 'completed_by']
for f in files:
    with open(mypath+f) as json_file:
        ground_truth = json.load(json_file)
        
        for i in range(len(ground_truth)):
            lista = ground_truth[i]['annotations'][0]['result']

            for l in lista:
                # acá salteo todos los que no tengan el formato que me interesa
                try:
                    campos.append(l['from_name'])
                except:
                    continue

cols = sorted(set(campos))
cols

['apoderado_1',
 'apoderado_2',
 'apoderado_3',
 'apoderado_4',
 'apoderado_5',
 'boleta_inscripcion_bb',
 'completed_by',
 'cuadro_accionario_bb',
 'datos_inscripcion',
 'denominacion_social',
 'documento',
 'duracion_sociedad',
 'fecha_inscripcion',
 'fecha_otorgamiento',
 'id',
 'instrumento_publico',
 'is_primera_pagina',
 'lugar_otorgamiento',
 'notaria_num',
 'notario',
 'orden_del_dia_bb',
 'pagina',
 'regimen_social',
 'registro_publico_correspondiente',
 'tipos_facultades']

In [67]:
df_truth = pd.DataFrame(columns = cols)

In [68]:
for f in files:
    df_tmp = pd.DataFrame(columns = cols)
    with open(mypath+f) as json_file:
        ground_truth = json.load(json_file)
        #print(len(ground_truth))

        for i in range(len(ground_truth)):
        #     print(i)
            identity = ground_truth[i]['id']
            documento = ground_truth[i]['file_upload'].split('-')[0]
            pagina = ground_truth[i]['file_upload'].split('-')[1].split('.')[0].split('_')[0]
            completed_by = ground_truth[i]['annotations'][0]['completed_by']['email']

            lista = ground_truth[i]['annotations'][0]['result']

            for l in lista:
                df_tmp.loc[i, 'id'] = identity
                df_tmp.loc[i, 'documento'] = documento
                df_tmp.loc[i, 'pagina'] = pagina
                df_tmp.loc[i, 'completed_by'] = completed_by

                # algunos campos aparecen en "text"
                try:
                    df_tmp.loc[i, l['from_name']] = l['value']['text'][0]
                except:
                    # otros aparecen en "choices"
                    try:
                        df_tmp.loc[i, l['from_name']] = l['value']['choices'][0]
                    except:
                        # y otros en "rectanglelabels"
                        try:
                            df_tmp.loc[i, l['from_name']] = l['value']['rectanglelabels'][0]
                        except:
                            continue
    df_truth = pd.concat([df_truth, df_tmp])
                    
# ordeno por documento y página
df_truth = df_truth.sort_values(['documento', 'pagina'])

# convierto algunos campos a int
df_truth['id'] = df_truth['id'].astype(int)
df_truth['documento'] = df_truth['documento'].astype(int)
df_truth['pagina'] = df_truth['pagina'].astype(int)

# pongo id, documento y pagina al ppio:
for c in ['pagina', 'documento', 'id']:
    first_column = df_truth.pop(c)
    df_truth.insert(0, c, first_column)

print(df_truth.info())
df_truth.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233 entries, 2 to 0
Data columns (total 25 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   id                                233 non-null    int64 
 1   documento                         233 non-null    int64 
 2   pagina                            233 non-null    int64 
 3   apoderado_1                       43 non-null     object
 4   apoderado_2                       19 non-null     object
 5   apoderado_3                       6 non-null      object
 6   apoderado_4                       3 non-null      object
 7   apoderado_5                       1 non-null      object
 8   boleta_inscripcion_bb             33 non-null     object
 9   completed_by                      233 non-null    object
 10  cuadro_accionario_bb              47 non-null     object
 11  datos_inscripcion                 35 non-null     object
 12  denominacion_social     

,id,documento,pagina,apoderado_1,apoderado_2,apoderado_3,apoderado_4,apoderado_5,boleta_inscripcion_bb,completed_by,...,fecha_otorgamiento,instrumento_publico,is_primera_pagina,lugar_otorgamiento,notaria_num,notario,orden_del_dia_bb,regimen_social,registro_publico_correspondiente,tipos_facultades
2,8774,10227,0,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,07/08/2017,5605,acta constitutiva,Nayarit,30,Ney González Sánchez,NaN,NaN,NaN,NaN
15,8780,10227,2,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S DE RL DE CV,NaN,NaN
16,8779,10227,3,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,8789,10227,4,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,acta constitutiva,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,8783,10227,6,NaN,NaN,NaN,NaN,NaN,NaN,nancy.campos@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,actos de administracion


## Distribución de páginas

In [69]:
paginas = {}

for column in df_truth.drop(['id', 'documento', 'pagina'], axis=1):
    paginas[column] = list(df_truth.loc[df_truth[column].notnull(), 'pagina'])

In [70]:
import plotly.express as px

for k in paginas.keys():
    try:
        fig = px.histogram(paginas[k], x=paginas[k], title=k, width=600, height=400)
        fig.show()
    except:
        continue

In [71]:
# cuántos documentos hay
len(pd.unique(df_truth['documento']))

42

## Busco el outlier que se vio arriba

In [72]:
#df_truth.head()

In [73]:
df_tmp = df_truth[['documento', 'pagina', 'instrumento_publico', 'completed_by']].loc[df_truth['instrumento_publico'].notnull()]
#df_tmp.head()

In [74]:
df_tmp.loc[df_tmp['pagina'] > 25]

,documento,pagina,instrumento_publico,completed_by
43,17513,26,81920,federico.rabinovich@konfio.mx


## Repito los gráficos de arriba, pero con las páginas como porcentaje

In [75]:
#f = files[0]
#with open(mypath+f) as json_file:
#    ground_truth = json.load(json_file)

#ground_truth[0]

In [76]:
with open('/Users/data/Documents/actas/entityid_pagenum.json') as json_file:
    dict_pages = json.load(json_file)

#dict_pages

In [77]:
df_truth = df_truth.reindex(columns = df_truth.columns.tolist() 
                                  + ['doc_length'])
#df_truth.head()

In [78]:
for k in dict_pages.keys():
    df_truth.loc[df_truth['documento']==int(k), 'doc_length'] = dict_pages[k]

In [79]:
# creo la columna con las páginas expresadas como porcentaje de la cantidad total de páginas
df_truth['page_ratio'] = df_truth['pagina']/df_truth['doc_length']

In [80]:
#df_truth[['documento', 'pagina', 'doc_length', 'page_ratio']].head()

### Gráficos

In [81]:
page_ratio_dist = {}

for column in df_truth.drop(['id', 'documento', 'pagina', 'doc_length', 'page_ratio', 'completed_by'], axis=1):
    page_ratio_dist[column] = list(df_truth.loc[df_truth[column].notnull(), 'page_ratio'])
    
#page_ratio_dist

In [82]:
import plotly.express as px

for k in page_ratio_dist.keys():
    try:
        fig = px.histogram(page_ratio_dist[k], x=page_ratio_dist[k], title=k, width=600, height=400)
        fig.show()
    except:
        continue

### Busco los casos que dieron un ratio >1

In [83]:
df_tmp = df_truth.loc[df_truth['page_ratio']>1]
df_tmp.head()

,id,documento,pagina,apoderado_1,apoderado_2,apoderado_3,apoderado_4,apoderado_5,boleta_inscripcion_bb,completed_by,...,is_primera_pagina,lugar_otorgamiento,notaria_num,notario,orden_del_dia_bb,regimen_social,registro_publico_correspondiente,tipos_facultades,doc_length,page_ratio
15,9189,13881,38,NaN,NaN,NaN,NaN,NaN,boleta_inscripcion,norma.lopez@konfio.mx,...,boleta de inscripcion,NaN,NaN,NaN,NaN,NaN,REGISTRO PUBLICO DE COMERCIO DE LA CIUDAD DE M...,NaN,31.0,1.225806
17,9160,13881,42,NaN,NaN,NaN,NaN,NaN,NaN,norma.lopez@konfio.mx,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.0,1.354839


La 2da fila no tiene ninguna información. Sin embargo la página 42 existe, así que pareciera que para ese documento, el número total de páginas es incorrecto.

## Distribución de cantidad de páginas por documento

In [84]:
fig = px.histogram(df_truth, x='doc_length', width=600, height=400)
fig.update_layout(xaxis_title="Cantidad de páginas", yaxis_title="Count")
fig.show()